In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import imageio
import os
import optuna

# Define the PINN model with Dropout for ODE u'(t) = cos(2*pi*t)
def create_pinn_model(dropout_rate, num_neurons):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(num_neurons, activation='tanh', input_shape=(1,)),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(num_neurons, activation='tanh'),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(1, activation=None)
    ])
    return model

# Physics-informed loss function
def physics_loss(model, t, omega1=1, omega2=1, omega3=0.001, omega4=0.001):
    with tf.GradientTape() as tape:
        tape.watch(t)
        u_pred = model(t, training=True)
    u_t = tape.gradient(u_pred, t)
    
    # 1. Loss từ phương trình vi phân (PDE loss)
    loss_eq = tf.reduce_mean((u_t - tf.cos(2 * np.pi * t))**2)
    
    # 2. Loss từ điều kiện ban đầu (Initial Condition loss)
    loss_ic = tf.reduce_mean(model(tf.zeros((1, 1)), training=True)**2)

    # 3. Loss từ dữ liệu thực nghiệm (Data loss)
    u_exact = (1 / (2 * np.pi)) * tf.sin(2 * np.pi * t)
    loss_data = tf.reduce_mean((u_pred - u_exact)**2)

    # 4. Loss từ điều kiện biên (Boundary Condition loss) [Có thể chỉnh sửa]
    t_bc = tf.constant([[1.0]])  # Điều kiện biên tại t = 1
    u_bc_pred = model(t_bc, training=True)
    u_bc_exact = tf.constant([[0.0]])  # Giá trị đúng tại biên (có thể cần chỉnh)
    loss_bc = tf.reduce_mean((u_bc_pred - u_bc_exact)**2)

    total_loss = omega1 * loss_eq + omega2 * loss_ic + omega3 * loss_data + omega4 * loss_bc
    return total_loss


# Training points
t_train = tf.reshape(tf.linspace(0.0, 1.0, 100), (-1, 1))

# Objective function for Optuna optimization
def objective(trial):
    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.5)
    num_neurons = trial.suggest_categorical('num_neurons', [8, 16, 32, 64, 128])
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)

    model = create_pinn_model(dropout_rate, num_neurons)
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    for epoch in range(2000):
        with tf.GradientTape() as tape:
            loss_pinn = physics_loss(model, t_train)
        gradients = tape.gradient(loss_pinn, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    final_loss = physics_loss(model, t_train).numpy()
    return final_loss

# Run Optuna optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

print('Best parameters:', study.best_params)

# Train final model with optimal parameters
best_params = study.best_params
model = create_pinn_model(best_params['dropout_rate'], best_params['num_neurons'])
optimizer = tf.keras.optimizers.Adam(learning_rate=best_params['learning_rate'])

# Training loop
num_epochs = 1000
img_dir = 'training_frames'
os.makedirs(img_dir, exist_ok=True)
frames = []

for epoch in range(num_epochs):
    with tf.GradientTape() as tape:
        loss_pinn = physics_loss(model, t_train)
    gradients = tape.gradient(loss_pinn, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    if epoch % 1 == 0:
        print(f'Epoch {epoch}/{num_epochs}, PINN Loss: {loss_pinn.numpy()}')

        t_test = tf.reshape(tf.linspace(0, 1, 100), (-1, 1))
        u_pred = model(t_test)
        u_exact = (1 / (2 * np.pi)) * np.sin(2 * np.pi * t_test.numpy())

        plt.figure(figsize=(10, 6))
        plt.plot(t_test.numpy(), u_pred.numpy(), label=f'PINN Epoch {epoch}')
        plt.plot(t_test.numpy(), u_exact, '--k', label='Exact Solution')
        plt.legend()
        plt.xlabel('t')
        plt.ylabel('u(t)')
        plt.title(f'PINN ODE Solution at Epoch {epoch}')
        plt.ylim(-0.3, 0.3)

        frame_path = f'{img_dir}/frame_{epoch}.png'
        plt.savefig(frame_path)
        frames.append(imageio.imread(frame_path))
        plt.close()

# Create and save GIF
imageio.mimsave('pinn_solution_evolution.gif', frames, duration=5)
print('GIF saved as pinn_solution_evolution.gif')



2025-03-11 09:00:11.640314: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-11 09:00:11.768393: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741658411.836478  115042 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741658411.854947  115042 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-11 09:00:11.968734: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Best parameters: {'dropout_rate': 0.0002037984254264491, 'num_neurons': 128, 'learning_rate': 0.00010385274525099984}
Epoch 0/1000, PINN Loss: 0.5060518383979797


/tmp/ipykernel_115042/21645127.py:109: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  frames.append(imageio.imread(frame_path))


Epoch 1/1000, PINN Loss: 0.5051500797271729
Epoch 2/1000, PINN Loss: 0.5055239796638489
Epoch 3/1000, PINN Loss: 0.504965603351593
Epoch 4/1000, PINN Loss: 0.505274772644043
Epoch 5/1000, PINN Loss: 0.5044108629226685
Epoch 6/1000, PINN Loss: 0.5052171349525452
Epoch 7/1000, PINN Loss: 0.5047990679740906
Epoch 8/1000, PINN Loss: 0.504916250705719
Epoch 9/1000, PINN Loss: 0.5050949454307556
Epoch 10/1000, PINN Loss: 0.5044780373573303
Epoch 11/1000, PINN Loss: 0.5048038959503174
Epoch 12/1000, PINN Loss: 0.5046060085296631
Epoch 13/1000, PINN Loss: 0.5047838091850281
Epoch 14/1000, PINN Loss: 0.5047279000282288
Epoch 15/1000, PINN Loss: 0.5057113170623779
Epoch 16/1000, PINN Loss: 0.504765510559082
Epoch 17/1000, PINN Loss: 0.5051983594894409
Epoch 18/1000, PINN Loss: 0.504909336566925
Epoch 19/1000, PINN Loss: 0.5050274133682251
Epoch 20/1000, PINN Loss: 0.504604697227478
Epoch 21/1000, PINN Loss: 0.5045807957649231
Epoch 22/1000, PINN Loss: 0.5054275393486023
Epoch 23/1000, PINN Loss: